In [ ]:
#default_exp core

# Python Wrapper for Codeforces API
> Wrapper around the codeforces api. Documentation for codeforces api can be found [here](https://codeforces.com/apiHelp)

## Importing Required Libraries

In [ ]:
#export
import datetime
import hashlib
import random
import string
import time

import requests

from co_py.types import *

## CFApi

> Base URL for the codeforces API - `https://codeforces.com/api/{method_name}`

The codeforces API supports two different languages: 'eu' and 'ru'. By default lang='eu' is set.

Some methods requires `api_key` and `secret` for authentication

In [ ]:
#export
class CFApi:
    _BASE_API = 'https://codeforces.com/api/'
    LANG = ('en', 'ru')
    
    def __init__(self, api_key: str = None, secret: str = None, lang='en'):
        self.api_key = api_key
        self.secret = secret
        self.lang = lang
        self.anonymous = True
        # check both key and secret are supplied
        if self.api_key and self.secret:
            self.anonymous = False
        # if one is provided and the other is not, raise Error
        if (self.api_key is None) ^ (self.secret is None):
            raise Exception('both key are secret are required')
        if self.lang not in self.LANG:
            raise Exception(f'{self.lang:} is not supported')
    
    
    def _generate_secret_params(self, method_name: str, params) -> dict:
        """Generates the parameters required for the authentication purpose"""
        cur_time = int(time.time())
        rand_options = string.printable[:62]
        rand_char = random.choices(rand_options, k=6)
        rand_char = "".join(rand_char)
        
        params['apiKey'] = self.api_key
        params['time'] = str(cur_time)

        sorted_params = {key: val for key, val in sorted(params.items())}  # only for Python3.6 and above
        params_encoded = requests.urllib3.request.urlencode(sorted_params, safe=';')
        
        api_sig = f'{rand_char}/{method_name}?{params_encoded}#{self.secret}'
        api_sig = api_sig.encode('utf-8')
        api_sig = hashlib.sha512(api_sig).hexdigest()
        secret_params = {
            'apiKey': self.api_key,
            'time': str(cur_time),
            'apiSig': rand_char + api_sig
        }
        return secret_params
    
    
    def _get_data(self, method_name: str, params: dict = None):
        url = self._BASE_API + method_name
        cur_time = datetime.datetime.now().timestamp()
        if params is None:
            params = {}
        
        if not self.anonymous:
            secret_params = self._generate_secret_params(method_name, params)
            params.update(secret_params)
            
        res = requests.get(url, params=params)
        if res.status_code != 200:
            raise Exception(f'{res.status_code}')
        res = res.json()
        if res['status'] != 'OK':
            raise Exception(res)
        return res['result']
    
    
    def get_contests(self, gym: bool = False):
        method_name = 'contest.list'
        params = {'gym': gym}
        contests = self._get_data(method_name, params)
        contests = [Contest(contest) for contest in contests]
        return contests
    
    
    def get_problems(self, tags=None):
        method_name = 'problemset.problems'
        if isinstance(tags, str):
            tags = [tags]
        if tags:
            tags = ";".join(tags)
        params = {'tags': tags}
        res = self._get_data(method_name, params)
        problems = [Problem(problem) for problem in res['problems']]
        return problems
    
    
    def get_user_info(self, handles):
        method_name = 'user.info'
        if isinstance(handles, str):
            handles = [handles]
        handles = ";".join(handles)
        params = {'handles': handles}
        users = self._get_data(method_name, params)
        users = [User(user) for user in users]
        return users
    
    
    def get_user_ratings(self, handle: str):
        method_name = 'user.rating'
        params = {'handle': handle}
        user_ratings = self._get_data(method_name, params)
        user_ratings = [UserRating(rating) for rating in user_ratings]
        return user_ratings
    
    
    def get_user_submissions(self, handle: str):
        # from and count
        method_name = 'user.status'
        params = {'handle': handle}
        submissions = self._get_data(method_name, params)
        submissions = [Submission(submission) for submission in submissions]
        return submissions
    
    
    def get_blog_comments(self, blog_id: int) -> Comment:
        method_name = 'blogEntry.comments'
        params = {'blogEntryId': blog_id}
        comments = self._get_data(method_name, params)
        comments = [Comment(comment) for comment in comments]
        return comments
    
    
    def get_blog_entry(self, blog_id: int) -> BlogEntry:
        method_name = 'blogEntry.view'
        params = {'blogEntryId': blog_id}
        blog = self._get_data(method_name, params)
        blog = BlogEntry(blog)
        return blog
    
    
    def get_contest_standings(self, contest_id: int):
        # from , count, show_unofficial remaining
        method_name = 'contest.standings'
        params = {'contestId': contest_id}
        standings = self._get_data(method_name, params)
        standings = Standings(standings)
        return standings
    
    
    def get_user_friends(self, online=False) -> list:
        method_name = 'user.friends'
        params = {'onlyOnline': online}
        friends = self._get_data(method_name, params)
        return friends
    
    def get_contest_hacks(self, contest_id):
        method_name = 'contest.hacks'
        params = {'contestId': contest_id}
        hacks = self._get_data(method_name, params)
        hacks = [Hack(hack) for hack in hacks]
        return hacks
    
    def get_contest_rating_changes(self, contest_id):
        method_name = 'contest.ratingChanges'
        params = {'contestId': contest_id}
        rating_changes = self._get_data(method_name, params)
        rating_changes = [RatingChange(r) for r in rating_changes]
        return rating_changes
    
    
    def get_recent_submissions(self, count=10):
        method_name = 'problemset.recentStatus'
        params = {'count': count}
        recent_subs = self._get_data(method_name, params)
        recent_subs = [Submission(sub) for sub in recent_subs]
        return recent_subs
    
    
    def get_recent_action(self, max_count=10):
        method_name = 'recentActions'
        params = {'maxCount': max_count}
        recent_action = self._get_data(method_name, params)
        recent_action = [RecentAction(r) for r in recent_action]
        return recent_action
    
    
    def get_user_blog_entry(self, handle):
        method_name = 'user.blogEntries'
        params = {'handle': handle}
        blog_entry = self._get_data(method_name, params)
        blog_entry = [BlogEntry(blog) for blog in blog_entry]
        return blog_entry
    
    
    def get_rating_list(self, active_only=False):
        method_name = 'user.ratedList'
        params = {'activeOnly': active_only}
        rating = self._get_data(method_name, params)
        rating = [User(user) for user in rating]
        return rating
    
    
    def get_contest_status(self, contest_id):
        # from, count, user
        method_name = 'contest.status'
        params = {'contestId': contest_id}
        submissions = self._get_data(method_name, params)
        subsmissons = [Submissions(subs) for subs in submissions]
        return submissions

## Tests

In [ ]:
#export
#hide
cf = CFApi()

In [ ]:
cf.lang

'en'

#### Get contest

In [ ]:
#export
#hide
contests = cf.get_contests()

In [ ]:
#export
#hide
contest = list(filter(lambda c: c.id == 1521, contests))[0]

In [ ]:
#export
#hide
assert isinstance(contest, Contest)

In [ ]:
#export
#hide
assert contest.id == 1521
assert contest.name == 'Codeforces Round #720 (Div. 2)'
assert contest.type == 'CF'
assert contest.phase == 'FINISHED'
assert contest.frozen == False
assert contest.duration_seconds == 8100

#### Problems

In [ ]:
problems = cf.get_problems()
assert isinstance(problems[0], Problem)

In [ ]:
problem = list(filter(lambda p: p.contest_id == 1521 and p.index == 'A', problems))[0]

In [ ]:
assert isinstance(problem, Problem)

In [ ]:
assert problem.contest_id == 1521
assert problem.index == 'A'
assert problem.name == 'Nastia and Nearly Good Numbers'
assert problem.type == 'PROGRAMMING'
assert problem.points == 500.0
assert problem.rating == 1000
assert problem.tags == ['constructive algorithms', 'math', 'number theory']

#### Get Blogs

In [ ]:
blog = cf.get_blog_entry('90603')

In [ ]:
assert isinstance(blog, BlogEntry)

In [ ]:
assert blog.id == 90603
assert blog.author_handle == 'geekypandey'
assert blog.creation_time_seconds == 1620696608
assert blog.title == '<p>Codeforces Tracker</p>'
assert blog.locale == 'en'

#### Blog Comments

In [ ]:
comments = cf.get_blog_comments(90603)

In [ ]:
comment = list(filter(lambda c: c.id == 790332, comments))[0]

In [ ]:
assert isinstance(comment, Comment)

In [ ]:
comment.text

'<div class="ttypography"><p><i>Auto comment: topic has been updated by <a class="rated-user user-gray" href="/profile/geekypandey" title="Новичок geekypandey">geekypandey</a> (<a href="/topic/91213/en3">previous revision</a>, <a href="/topic/91213/en4">new revision</a>, <a href="/topic/91213/diff/en3/en4">compare</a>).</i></p></div>'

In [ ]:
assert comment.id == 790332
assert comment.creation_time_seconds == 1620704584
assert comment.commentator_handle == 'geekypandey'
assert comment.locale == 'en'
assert comment.text == '<div class="ttypography"><p><i>Auto comment: topic has been updated by <a class="rated-user user-gray" href="/profile/geekypandey" title="Новичок geekypandey">geekypandey</a> (<a href="/topic/91213/en3">previous revision</a>, <a href="/topic/91213/en4">new revision</a>, <a href="/topic/91213/diff/en3/en4">compare</a>).</i></p></div>'
assert comment.parent_comment_id is None

#### Contest hacks

500 status_code check

In [ ]:
hacks = cf.get_contest_hacks(1521)

In [ ]:
hack = list(filter(lambda x: x.id == 736732, hacks))[0]

In [ ]:
hack

{'id': 736732, 'creation_time_seconds': 1620399713, 'hacker': {'contest_id': 1521, 'members': [{'handle': 'codeguptaji'}], 'participant_type': 'CONTESTANT', 'team_id': None, 'team_name': None, 'ghost': False, 'room': 456, 'start_time_seconds': 1620398100}, 'defender': {'contest_id': 1521, 'members': [{'handle': 'psycho437'}], 'participant_type': 'CONTESTANT', 'team_id': None, 'team_name': None, 'ghost': False, 'room': 456, 'start_time_seconds': 1620398100}, 'verdict': 'INVALID_INPUT', 'problem': {'contest_id': 1521, 'problemset_name': None, 'index': 'A', 'name': 'Nastia and Nearly Good Numbers', 'type': 'PROGRAMMING', 'points': 500.0, 'rating': 1000, 'tags': ['constructive algorithms', 'math', 'number theory']}, 'test': None, 'judge_protocol': {'protocol': "Validator 'validator.exe' returns exit code 3 [FAIL Expected EOLN (stdin, line 1)]", 'manual': 'false', 'verdict': 'Invalid input'}}

In [ ]:
assert isinstance(hack, Hack)

In [ ]:
assert hack.id == 736732
assert hack.creation_time_seconds == 1620399713
assert isinstance(hack.hacker, Party)
assert hack.verdict == 'INVALID_INPUT'
assert isinstance(hack.problem, Problem)
assert hack.problem.rating == 1000
assert hack.problem.tags == ['constructive algorithms', 'math', 'number theory']

In [ ]:
from pprint import pprint
pprint(hack)

{'id': 736732, 'creation_time_seconds': 1620399713, 'hacker': {'contest_id': 1521, 'members': [{'handle': 'codeguptaji'}], 'participant_type': 'CONTESTANT', 'team_id': None, 'team_name': None, 'ghost': False, 'room': 456, 'start_time_seconds': 1620398100}, 'defender': {'contest_id': 1521, 'members': [{'handle': 'psycho437'}], 'participant_type': 'CONTESTANT', 'team_id': None, 'team_name': None, 'ghost': False, 'room': 456, 'start_time_seconds': 1620398100}, 'verdict': 'INVALID_INPUT', 'problem': {'contest_id': 1521, 'problemset_name': None, 'index': 'A', 'name': 'Nastia and Nearly Good Numbers', 'type': 'PROGRAMMING', 'points': 500.0, 'rating': 1000, 'tags': ['constructive algorithms', 'math', 'number theory']}, 'test': None, 'judge_protocol': {'protocol': "Validator 'validator.exe' returns exit code 3 [FAIL Expected EOLN (stdin, line 1)]", 'manual': 'false', 'verdict': 'Invalid input'}}


#### Contest Rating Changes

In [ ]:
contest_rating_changes = cf.get_contest_rating_changes(1511)

In [ ]:
contest_rating_changes[0]

{'contest_id': 1511, 'contest_name': 'Educational Codeforces Round 107 (Rated for Div. 2)', 'handle': 'Ideal_End', 'rank': 1, 'rating_update_time_seconds': 1618245300, 'old_rating': 924, 'new_rating': 1619}

In [ ]:
user_rating_change = list(filter(lambda x: x.handle == 'geekypandey', contest_rating_changes))[0]

In [ ]:
user_rating_change

{'contest_id': 1511, 'contest_name': 'Educational Codeforces Round 107 (Rated for Div. 2)', 'handle': 'geekypandey', 'rank': 9024, 'rating_update_time_seconds': 1618245300, 'old_rating': 1174, 'new_rating': 1111}

In [ ]:
assert user_rating_change.contest_id == 1511
assert user_rating_change.contest_name == 'Educational Codeforces Round 107 (Rated for Div. 2)'
assert user_rating_change.handle == 'geekypandey'
assert user_rating_change.rank == 9024
assert user_rating_change.old_rating == 1174
assert user_rating_change.new_rating == 1111

#### Contest Standings

In [ ]:
standings = cf.get_contest_standings(1521)